# Deep Learning with Weather Data

### In this notebox a temperature forecast is calculated with the help of a fully connected network. The model is based on freely available data from the German Weather Service (DWD) and Open Sense Box data.

In [13]:
from arcgis.gis import GIS
from arcgis import GIS,GeoSeriesAccessor, GeoAccessor
import pandas as pd
import ftplib
import requests
import zipfile
from io import BytesIO
import gzip
import arcpy
from datetime import datetime, timedelta
import arcgis
import arcgis.raster
from arcgis.raster import RasterManager
from arcgis.raster import RasterCatalogItem
# delete irrelevant files
from pathlib import Path
from urllib.parse import urlparse
import re
import os
gis = GIS("home")

#### 1. : Get DWD Data

In [3]:
p = Path("C:/Data/DWD/Berlin_dwd/Temp/")

In [4]:
import requests

url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/P0_Stundenwerte_Beschreibung_Stationen.txt"

#just a random link of a dummy file
r = requests.get(url)
#retrieving data from the URL using get method
with open("C:\Data\DWD\Berlin_dwd\Temp\stations.txt", 'wb') as f:
#giving a name and saving it in any required format
#opening the file in write mode
    f.write(r.content) 
#writes the URL contents from the server

In [5]:
import csv

with open("C:\Data\DWD\Berlin_dwd\Temp\stations.txt", 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open("C:\Data\DWD\Berlin_dwd\Temp\stations.csv", 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('title', 'intro'))
        writer.writerows(lines)

In [6]:
# Get Longitude and Latitude of the station
table_csv_path = r"C:\Data\DWD\dwd.xls" # hier noch python download einfügen
stationdf =pd.read_excel(table_csv_path)
stationdf=stationdf.set_index(['Stations_id'])
#stationdf

In [45]:
# With stationid load only data of interest
Berlin = ["00303","00400","00403","00420","00427","00433","02856","03376","03987","05825","07389","03015"]


for i in Berlin:
    mytempURL ="https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/recent/stundenwerte_TU_"+i+"_akt.zip"
    #print(mytempURL)
    filename = mytempURL.split('/')[-1]
    req = requests.get(url = mytempURL, stream = False)
    print('Downloading Completed')
    print(req)
        
    # extracting the zip file contents
    zipf= zipfile.ZipFile(BytesIO(req.content))
    zipf.extractall(r'C:\Data\DWD\Berlin_dwd\Temp')
    
# Delete unimportant files    
for file_name in p.glob('*Metadaten*'):
    os.remove(file_name)
    #print('Deleting file:', file_name)

dft_list= []

#Create a dft list
for file_name in p.glob('*produkt_tu*'):
    print(file_name)
    
    dft = pd.read_csv(file_name, sep=";")
    dft=dft.set_index(['STATIONS_ID'])
    dft2022=pd.merge(dft,stationdf,  left_index=True, right_index=True)
    dft2022.index.names = ['Stations_id']
    dft2022.reset_index(inplace = True)
    
    dft_list.append(dft2022)
    
# Create one Dataframe of all elements in df_list
for dft in dft_list:
    dft2022 = dft2022.append([dft])

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00303.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00400.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00403.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00420.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00427.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00433.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_02856.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03015.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03376.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03987.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_2

In [7]:
BerlinPressure = ["00303","00403","00427","00433","03987","03015"]#Die anderen Stationen haben keine pressure Werte

for i in BerlinPressure:
    mypressureURL ="https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_"+i+"_akt.zip"
    print(mypressureURL)
    filename = mypressureURL.split('/')[-1]
    req = requests.get(url = mypressureURL, stream = False)
    print('Downloading Completed')
    print(req)
        
    # extracting the zip file contents
    zipf= zipfile.ZipFile(BytesIO(req.content))
    zipf.extractall(r'C:\Data\DWD\Berlin_dwd\Temp')
    
for file_name in p.glob('*Metadaten*'):
    os.remove(file_name)
    #print('Deleting file:', file_name)  
    

dfp_list= []

for file_name in p.glob('*produkt_p0*'):
    print(file_name)
    
    dfp = pd.read_csv(file_name, sep=";")
    dfp=dfp.set_index(['STATIONS_ID'])
    dfp2022=pd.merge(dfp,stationdf,  left_index=True, right_index=True)
    dfp2022.index.names = ['Stations_id']
    dfp2022.reset_index(inplace = True)
    
    dfp_list.append(dfp2022)
    
# Create one Dataframe of all elements in df_list
for dfp in dfp_list:
    dfp2022 = dfp2022.append([dfp])

https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_00303_akt.zip
<Response [200]>
https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_00403_akt.zip
<Response [200]>
https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_00427_akt.zip
<Response [200]>
https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_00433_akt.zip
<Response [200]>
https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_03987_akt.zip
<Response [200]>
https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/pressure/recent/stundenwerte_P0_03015_akt.zip
<Response [200]>
C:\Data\DWD\Berlin_dwd\Temp\produkt_p0_stunde_20210624_20221225_00303.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_p0_stund

In [8]:
# only even numbers. 3, 9, 15, 21 are only up to 33h in advance
Berlin = ["00303","00400","00403","00420","00427","00433","02856","03376","03987","05825","07389","03015"]


for i in Berlin:
    mytempURL ="https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/recent/stundenwerte_TU_"+i+"_akt.zip"
    #print(mytempURL)
    filename = mytempURL.split('/')[-1]
    req = requests.get(url = mytempURL, stream = False)
    print('Downloading Completed')
    print(req)
        
    # extracting the zip file contents
    zipf= zipfile.ZipFile(BytesIO(req.content))
    zipf.extractall(r'C:\Data\DWD\Berlin_dwd\Temp')
    
for file_name in p.glob('*Metadaten*'):
    os.remove(file_name)
    #print('Deleting file:', file_name)

dft_list= []

#Create a dft list
for file_name in p.glob('*produkt_tu*'):
    print(file_name)
    
    dft = pd.read_csv(file_name, sep=";")
    dft=dft.set_index(['STATIONS_ID'])
    dft2022=pd.merge(dft,stationdf,  left_index=True, right_index=True)
    dft2022.index.names = ['Stations_id']
    dft2022.reset_index(inplace = True)
    
    dft_list.append(dft2022)
    
# Create one Dataframe of all elements in df_list
for dft in dft_list:
    dft2022 = dft2022.append([dft])

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00303.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00400.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00403.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00420.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00427.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_00433.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_02856.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03015.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03376.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_20221225_03987.txt
C:\Data\DWD\Berlin_dwd\Temp\produkt_tu_stunde_20210624_2

In [9]:
df2022=pd.merge(dfp2022,dft2022)
df2022

,Stations_id,MESS_DATUM,QN_8,P,P0,eor,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,QN_9,TT_TU,RF_TU
0,3987,2021071700,3,1018.4,1006.8,eor,18930101,20221120,82,52.3812,13.0622,Potsdam,Brandenburg,3,20.7,98.0
1,3987,2021071700,3,1018.4,1006.8,eor,18930101,20221120,82,52.3812,13.0622,Potsdam,Brandenburg,3,20.7,98.0
2,3987,2021071700,3,1018.4,1006.8,eor,18930101,20221120,82,52.3812,13.0622,Potsdam,Brandenburg,3,20.7,98.0
3,3987,2021071700,3,1018.4,1006.8,eor,18930101,20221120,82,52.3812,13.0622,Potsdam,Brandenburg,3,20.7,98.0
4,3987,2021071700,3,1018.4,1006.8,eor,18930101,20221120,82,52.3812,13.0622,Potsdam,Brandenburg,3,20.7,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504247,3015,2023011719,1,990.7,978.0,eor,19510101,20221120,98,52.2085,14.1180,Lindenberg,Brandenburg,1,0.2,90.0
504248,3015,2023011720,1,990.6,977.9,eor,19510101,20221120,98,52.2085,14.1180,Lindenberg,Brandenburg,1,0.0,90.0
504249,3015,2023011721,1,991.5,978.7,eor,19510101,20221120,98,52.2085,14.1180,Lindenberg,Brandenburg,1,-1.0,91.0
504250,3015,2023011722,1,992.3,979.5,eor,19510101,20221120,98,52.2085,14.1180,Lindenberg,Brandenburg,1,-0.3,87.0


In [10]:
for col in df2022.columns:
    print(col)

Stations_id
MESS_DATUM
QN_8
   P
  P0
eor
von_datum
bis_datum
Stationshoehe
geoBreite
geoLaenge
Stationsname
Bundesland
QN_9
TT_TU
RF_TU


In [11]:
#TT_TU Lufttemperatur in 2m Höhe
#RF_TU relative Feuchte
#P Luftdruck auf Meereshöhe NN hPA
#P0 Luftdruck auf Stationshöhe
df2022=df2022.rename(columns={'Stations_id':'ID','MESS_DATUM':'date','geoBreite':'Latitude','geoLaenge':'Longitude','Messdatum':'date','TT_TU':'Temperatur','RF_TU':'relLuftfeuchte', '   P':'Luftdruck'})
df2022 = df2022.drop(['QN_8','  P0','eor','von_datum','bis_datum','Stationshoehe','Stationsname','Bundesland','QN_9'], axis=1)
df2022

,ID,date,Luftdruck,Latitude,Longitude,Temperatur,relLuftfeuchte
0,3987,2021071700,1018.4,52.3812,13.0622,20.7,98.0
1,3987,2021071700,1018.4,52.3812,13.0622,20.7,98.0
2,3987,2021071700,1018.4,52.3812,13.0622,20.7,98.0
3,3987,2021071700,1018.4,52.3812,13.0622,20.7,98.0
4,3987,2021071700,1018.4,52.3812,13.0622,20.7,98.0
...,...,...,...,...,...,...,...
504247,3015,2023011719,990.7,52.2085,14.1180,0.2,90.0
504248,3015,2023011720,990.6,52.2085,14.1180,0.0,90.0
504249,3015,2023011721,991.5,52.2085,14.1180,-1.0,91.0
504250,3015,2023011722,992.3,52.2085,14.1180,-0.3,87.0


### Get Opensensemap Data

In [14]:
# Anmeldung im GIS
agol = GIS("home")

In [15]:
# URL der API im JSON-Format
base_url = 'https://api.opensensemap.org/boxes?&full=true'

In [16]:
# Abfrage des Zeitstempels von vor 5 Minuten
timestamp = datetime.utcnow() - timedelta(minutes=15)


In [17]:
# Abfrage der SenseBoxen deren Messdaten maximal 5 Minuten alt sind
url = base_url +'&date='+str(timestamp.isoformat("T") + "Z") +'&bbox='+str(11.839449) +','+str(51.866560)+','+str(15.182040) +','+str(53.006758)

In [19]:
%%time
# Uebertragung der in der API abgefragten JSON in einen Dataframe
# Abfrage URL, Übertragung in PD Dataframe
try:
    df = pd.read_json(url,encoding='utf-8')
except:
    raise SystemExit('ERROR: Fehlerhafte Abfrage der OpenSense API!')

Wall time: 7.84 s


In [20]:
if df.shape[0] > 0:
    print('Die Abfrage der OpenSense API war erfolgreich.')
else:
    raise SystemExit('ERROR: Fehlerhafte Abfrage der OpenSense API!')

Die Abfrage der OpenSense API war erfolgreich.


In [21]:
# Da die API fuer die selben Phaenomene unterschiedliche Schlagwoerter haben kann, muessen Listen erstellt werden mit allen eingegebenen Schlagwoertern zum jeweiligen Phaenomen
# Bsp Temperatur: Temperatur, Lufttemperatur, temperature, usw. geben alle das selbe Phaenomen wieder

# Zugang zu der "sensors"-column und in das jeweilige Phaenomen
phenomenom_array = []
for index, row in df.iterrows():
    for i in row['sensors']:
# Versucche jeden "Titel" der Phaenomene der Liste hinzuzufuegen
        try:
            phenomenom_array.append(i['title'])
        except KeyError:
            continue
phenomenom_list = list(set(phenomenom_array))

In [22]:
# Erstellen verschiedener Phaenomen-Listen aus der phaenomen_list (aller vorhandenen Phaenomene)
luftfeuchte = [s for s in phenomenom_list if "Luftfeuchte" in s] + [s for s in phenomenom_list if "Humidity" in s] + [s for s in phenomenom_list if "Luftfeuchtigkeit" in s]
temperatur = [s for s in phenomenom_list if "Temperatur" in s] + [s for s in phenomenom_list if "Temperature" in s] + [s for s in phenomenom_list if "Lufttemperatur" in s]
luftdruck = [s for s in phenomenom_list if "Luftdruck" in s] + [s for s in phenomenom_list if "Pressure" in s]
beleuchtungsstaerke = [s for s in phenomenom_list if "Beleuchtungsstärke" in s] + [s for s in phenomenom_list if "Helligkeit" in s] + [s for s in phenomenom_list if "light intesity" in s]
uv_intesitaet = [s for s in phenomenom_list if "UV Intensitaet" in s] + [s for s in phenomenom_list if "UV" in s] + [s for s in phenomenom_list if "UV intensity" in s]
pm10 = [s for s in phenomenom_list if "pm10" in s] + [s for s in phenomenom_list if "PM10" in s]
pm25 = [s for s in phenomenom_list if "pm2.5" in s] + [s for s in phenomenom_list if "PM25" in s] + [s for s in phenomenom_list if "PM2.5" in s]

In [23]:
# Umstrukturierung des Pandas Dataframes in ein enpsrechendes/ansprechendes Format, so dass jedes Phaenomen und dessen Wert in einer eigenen Spalte steht
data = []
for index, row in df.iterrows():
# Zugang und Speicherung der verschiedenen columns in eigene Vareiablen um diese dem Dataframe spaeter wieder hinzuzufuegen
    #timestamp = df.loc[index,'currentLocation']['timestamp']
    # Longitude abfragen
    lon = df.loc[index,'currentLocation']['coordinates'][0]
    # Latitude abfragen
    lat = df.loc[index,'currentLocation']['coordinates'][1]
    id_ = df.loc[index,'_id']
    name_ = df.loc[index,'name']
    grouptag_ = str(df.loc[index,'grouptag'])
    exposure_ = df.loc[index,'exposure']
    model_ = df.loc[index,'model']
    description_ = str(df.loc[index,'description'])
    weblink_ = str(df.loc[index,'weblink'])
    image_ = str(df.loc[index,'image'])
    
#     for element in df.loc[index,'sensors']:
#         if element['lastMeasurement']:
#             lastMeasurementAt_ = element['lastMeasurement']['createdAt']
#             break
#         else: 
#             lastMeasurementAt_ = None

    for a in range(len(df.loc[index, 'sensors'])):
        try:
            lastMeasurementAt_ = df.loc[index, 'sensors'][a]['lastMeasurement']['createdAt']
            if lastMeasurementAt_ != None:
                break
            else:
                lastMeasurementAt_ == None
        except:
            lastMeasurementAt_ == None
            
#     try:
#         # Zeit der letzten Messung des ersten verfeugbaren Sensors abfragen, falls dieser vorhanden ist.
#         lastMeasurementAt_ = df.loc[index,'sensors'][0]['lastMeasurement']['createdAt']
#         #lastMeasurementAt_ = df.loc[index,'lastMeasurementAt']
#             # Column 'LastMeasurementAt' ist nicht bei jedem Sensor vorhanden, daher nehmen wir 
#             # df.loc[index,'sensors'][0]['lastMeasurement']['createdAt'], da dieser Wert für jeden Sensor verfügbar ist
#     except:
#         lastMeasurementAt_ = None
    
    # Abfragen der Werte ('value') der Phaenomene
    sensor_data={}
    for i in row['sensors']:
        try:
            # Zugang zu jedem einzelnen Value der Sensordaten, befuellen des leeren sensor_data dictionaries
            sensor_data[i['title']]=round(float(i['lastMeasurement']['value']),2)
        except Exception as E:
            continue
    # das bisher leere Data-Dictionary wird mit den entsprechenden Werten befuellt:
    try:
        data.append(
            {
            'ID': id_[:49],
            'Name': name_[:249],
            'Longitude': lon,
            'Latitude':  lat,
            'Grouptag': grouptag_[:249],
            'Exposure': exposure_[:249],
            'LetzteMessung':lastMeasurementAt_,
            # bei den Phaenomen values: es wird nach den Schlagwoertern der oben erstellten temperatur, luftfeuchte, etc Listen gesucht
            'Temperatur': next((sensor_data.get(key) for key in temperatur if key in sensor_data), None),
            'relLuftfeuchte' : next((sensor_data.get(key) for key in luftfeuchte if key in sensor_data), None),
            'Luftdruck': next((sensor_data.get(key) for key in luftdruck if key in sensor_data), None),
            'Beleuchtungsstärke': next((sensor_data.get(key) for key in beleuchtungsstaerke if key in sensor_data), None),
            'UVIntensität': next((sensor_data.get(key) for key in uv_intesitaet if key in sensor_data), None),
            'PM10': next((sensor_data.get(key) for key in pm10 if key in sensor_data), None),
            'PM25': next((sensor_data.get(key) for key in pm25 if key in sensor_data), None),
            'Model': model_[:249],
            'Description': description_[:249],
            'Weblink': weblink_[:249],
            'Image': image_[:249],
            }
        )
    except KeyError:
        continue

# Erstellen des Pandas Dataframes aus dem sensor dictionary
df_out=pd.DataFrame(data)
# Column "Letzte Messung" ins Datetime-Format wandeln
df_out['LetzteMessung'] = df_out['LetzteMessung'].astype('datetime64[ns]')
# Da aus der OpenSenseAPI faelschlicherweise aeltere Daten abgefragt werden, werden diese geloescht
up_date_15minutes = datetime.utcnow() - timedelta(minutes=15)
df_out = df_out[~(df_out['LetzteMessung'] < up_date_15minutes)]

In [24]:
from datetime import datetime
#df_out=df_out.drop(["Name", "Description","Weblink", "Image"], axis=1)
#df_out['LetzteMessung']=pd.to_datetime(df_out['LetzteMessung'])
#df_out['LetzteMessung'] = pd.to_datetime(df_out['LetzteMessung'],format='%d%m%Y%hh')
d=df_out['LetzteMessung'].astype(str)
df_out['date'] = d.str[:4]+d.str[5:7]+d.str[8:10]+d.str[11:13]
df_out = df_out.drop(['Name','Grouptag','Exposure','LetzteMessung','Beleuchtungsstärke','Model','Description','Weblink','Image'], axis=1)
df_out

,ID,Longitude,Latitude,Temperatur,relLuftfeuchte,Luftdruck,UVIntensität,PM10,PM25,date
0,582473be40198a001010e1a3,13.302834,52.457219,-3.36,99.99,999.21,101.25,1.80,1.70,2023011908
1,5889cb92ff876d00101be9dd,13.380978,52.465227,0.73,79.14,999.63,360.00,NaN,NaN,2023011908
2,58fb1549635722001156933b,13.164529,52.188587,-129.25,99.90,65468.00,NaN,12.35,10.57,2023011908
3,592ca4b851d3460011ea2635,13.392964,52.510969,20.90,68.90,NaN,NaN,16.23,10.93,2023011908
4,5936c52aad0fa30011033d67,13.326400,52.450577,-0.60,99.90,NaN,NaN,15.90,8.33,2023011908
...,...,...,...,...,...,...,...,...,...,...
156,63981bc2a7390d001b0b5c4e,12.586415,52.140614,-1.47,83.01,NaN,NaN,NaN,NaN,2023011908
157,6398c664a7390d001b5e0742,13.381320,52.458560,-2.71,84.12,NaN,39.38,6.40,2.60,2023011908
158,639de5160d9dfc001b7dcd4e,13.400380,52.398106,-4.55,92.84,100298.47,NaN,27.48,13.35,2023011908
159,63b9935ded063c0007963f21,13.270599,52.510769,-1.50,94.70,NaN,NaN,9.13,6.70,2023011908


In [25]:
df=pd.concat([df2022,df_out])
df

,ID,date,Luftdruck,Latitude,Longitude,Temperatur,relLuftfeuchte,UVIntensität,PM10,PM25
0,3987,2021071700,1018.40,52.381200,13.062200,20.70,98.00,NaN,NaN,NaN
1,3987,2021071700,1018.40,52.381200,13.062200,20.70,98.00,NaN,NaN,NaN
2,3987,2021071700,1018.40,52.381200,13.062200,20.70,98.00,NaN,NaN,NaN
3,3987,2021071700,1018.40,52.381200,13.062200,20.70,98.00,NaN,NaN,NaN
4,3987,2021071700,1018.40,52.381200,13.062200,20.70,98.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
156,63981bc2a7390d001b0b5c4e,2023011908,NaN,52.140614,12.586415,-1.47,83.01,NaN,NaN,NaN
157,6398c664a7390d001b5e0742,2023011908,NaN,52.458560,13.381320,-2.71,84.12,39.38,6.40,2.60
158,639de5160d9dfc001b7dcd4e,2023011908,100298.47,52.398106,13.400380,-4.55,92.84,NaN,27.48,13.35
159,63b9935ded063c0007963f21,2023011908,NaN,52.510769,13.270599,-1.50,94.70,NaN,9.13,6.70


In [26]:
# Erstellen eines Spatially Enabled Dataframes
#sdf = GeoAccessor.from_xy(df,'Longitude', 'Latitude', sr=4326)
#sdf